<a href="https://colab.research.google.com/github/Geoffrey0pv/Inventory-management/blob/main/Inventory_Managment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Clasificación ABC de SKUS***

# Organizar Data Base

In [586]:
import numpy as np
import pandas as pd
import io

In [587]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [588]:
data = pd.read_excel("/content/drive/MyDrive/Operaciones I/Base de datos/Datos Ope1 Team 3 (1).xlsx")

In [589]:
data

,Tipo,Familias,SKUS,Cargo de Mantener una unidad en un año,Rentabilidad,Total Demanda,Promedio semana,Desv. Estándar,Cv,Costo unitario,Costo Ordenar,Tasa Pdn (unid por semana)
0,compra y venta,Familia 1,SKU 1,0.18,0.18,100571.0,1117.455556,548.693672,0.491021,26478.0,3415860.0,NaN
1,compra y venta,NaN,SKU 2,0.18,0.25,108196.0,1202.177778,598.309990,0.497688,35621.0,5124610.0,NaN
2,compra y venta,NaN,SKU 3,0.18,0.22,106396.0,1182.177778,603.645267,0.510621,40218.0,2471600.0,NaN
3,Manufacturado,Familia 2,SKU 4,0.26,0.26,131526.0,1461.400000,274.140774,0.187588,64570.0,3451200.0,1800.0
4,Manufacturado,NaN,SKU 5,0.26,0.25,128338.0,1425.977778,253.785275,0.177973,56478.0,4023600.0,2000.0
5,Manufacturado,NaN,SKU 6,0.26,0.3,131255.0,1458.388889,281.232548,0.192838,34867.0,3845650.0,1700.0
6,compra y venta,Familia 3,SKU 7,0.20,0.15,120159.0,1335.100000,456.989761,0.342289,56416.0,6451870.0,NaN
7,compra y venta,NaN,SKU 8,0.20,0.21,123184.0,1368.711111,501.595393,0.366473,64179.0,4781900.0,NaN
8,compra y venta,NaN,SKU 9,0.20,0.25,120719.0,1341.322222,413.133431,0.308005,42351.0,2345640.0,NaN
9,Manufacturado,Familia 4,SKU 10,0.25,0.28,113177.0,1257.522222,434.914669,0.345850,62415.0,7845000.0,1500.0


In [590]:
data.columns

Index(['Tipo', 'Familias', 'SKUS', 'Cargo de Mantener una unidad en un año',
       'Rentabilidad', 'Total Demanda', 'Promedio semana', 'Desv. Estándar',
       'Cv', 'Costo unitario ', 'Costo Ordenar', 'Tasa Pdn (unid por semana)'],
      dtype='object')

In [591]:
data = data.drop([12,13,14])

In [592]:
data.shape

(12, 12)

# Clasificación ABC - Ventas

## Clasificación ABC costo unitario (Demanda x costo)


In [593]:
dataSubset = data[["SKUS",'Total Demanda','Costo unitario ']]

In [594]:
dataSubset

,SKUS,Total Demanda,Costo unitario
0,SKU 1,100571.0,26478.0
1,SKU 2,108196.0,35621.0
2,SKU 3,106396.0,40218.0
3,SKU 4,131526.0,64570.0
4,SKU 5,128338.0,56478.0
5,SKU 6,131255.0,34867.0
6,SKU 7,120159.0,56416.0
7,SKU 8,123184.0,64179.0
8,SKU 9,120719.0,42351.0
9,SKU 10,113177.0,62415.0


In [595]:
dataSubset["Valor Total costos por sku"] = dataSubset["Total Demanda"] * dataSubset['Costo unitario ']

<ipython-input-595-ed143b097362>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSubset["Valor Total costos por sku"] = dataSubset["Total Demanda"] * dataSubset['Costo unitario ']


In [596]:
dataSubset = dataSubset.sort_values(by='Valor Total costos por sku', ascending=False)

In [597]:
totalCosts = dataSubset["Valor Total costos por sku"].sum()
totalCosts

64651600203.0

In [598]:
dataSubset

,SKUS,Total Demanda,Costo unitario,Valor Total costos por sku
3,SKU 4,131526.0,64570.0,8.492634e+09
7,SKU 8,123184.0,64179.0,7.905826e+09
4,SKU 5,128338.0,56478.0,7.248274e+09
9,SKU 10,113177.0,62415.0,7.063942e+09
6,SKU 7,120159.0,56416.0,6.778890e+09
8,SKU 9,120719.0,42351.0,5.112570e+09
5,SKU 6,131255.0,34867.0,4.576468e+09
2,SKU 3,106396.0,40218.0,4.279034e+09
10,SKU 11,112246.0,35468.0,3.981141e+09
1,SKU 2,108196.0,35621.0,3.854050e+09


In [599]:
dataSubset["Costos Acumuludos"] = dataSubset["Valor Total costos por sku"].cumsum()

In [600]:
dataSubset

,SKUS,Total Demanda,Costo unitario,Valor Total costos por sku,Costos Acumuludos
3,SKU 4,131526.0,64570.0,8.492634e+09,8.492634e+09
7,SKU 8,123184.0,64179.0,7.905826e+09,1.639846e+10
4,SKU 5,128338.0,56478.0,7.248274e+09,2.364673e+10
9,SKU 10,113177.0,62415.0,7.063942e+09,3.071068e+10
6,SKU 7,120159.0,56416.0,6.778890e+09,3.748957e+10
8,SKU 9,120719.0,42351.0,5.112570e+09,4.260214e+10
5,SKU 6,131255.0,34867.0,4.576468e+09,4.717860e+10
2,SKU 3,106396.0,40218.0,4.279034e+09,5.145764e+10
10,SKU 11,112246.0,35468.0,3.981141e+09,5.543878e+10
1,SKU 2,108196.0,35621.0,3.854050e+09,5.929283e+10


In [601]:
dataSubset["Porcentaje de costos acumulados"] = dataSubset['Costos Acumuludos'] / totalCosts

In [602]:
dataSubset

,SKUS,Total Demanda,Costo unitario,Valor Total costos por sku,Costos Acumuludos,Porcentaje de costos acumulados
3,SKU 4,131526.0,64570.0,8.492634e+09,8.492634e+09,0.131360
7,SKU 8,123184.0,64179.0,7.905826e+09,1.639846e+10,0.253644
4,SKU 5,128338.0,56478.0,7.248274e+09,2.364673e+10,0.365756
9,SKU 10,113177.0,62415.0,7.063942e+09,3.071068e+10,0.475018
6,SKU 7,120159.0,56416.0,6.778890e+09,3.748957e+10,0.579871
8,SKU 9,120719.0,42351.0,5.112570e+09,4.260214e+10,0.658949
5,SKU 6,131255.0,34867.0,4.576468e+09,4.717860e+10,0.729736
2,SKU 3,106396.0,40218.0,4.279034e+09,5.145764e+10,0.795922
10,SKU 11,112246.0,35468.0,3.981141e+09,5.543878e+10,0.857501
1,SKU 2,108196.0,35621.0,3.854050e+09,5.929283e+10,0.917113


In [603]:
def ABC(cumPer):
    if cumPer >= 0 and cumPer < 0.50:
        return 'A'
    elif cumPer >= 0.50 and cumPer <= 0.90:
        return 'B'
    elif cumPer > 0.90:
        return 'C'

In [604]:
dataSubset['Class'] = dataSubset['Porcentaje de costos acumulados'].apply(ABC)

In [605]:
dataSubset

,SKUS,Total Demanda,Costo unitario,Valor Total costos por sku,Costos Acumuludos,Porcentaje de costos acumulados,Class
3,SKU 4,131526.0,64570.0,8.492634e+09,8.492634e+09,0.131360,A
7,SKU 8,123184.0,64179.0,7.905826e+09,1.639846e+10,0.253644,A
4,SKU 5,128338.0,56478.0,7.248274e+09,2.364673e+10,0.365756,A
9,SKU 10,113177.0,62415.0,7.063942e+09,3.071068e+10,0.475018,A
6,SKU 7,120159.0,56416.0,6.778890e+09,3.748957e+10,0.579871,B
8,SKU 9,120719.0,42351.0,5.112570e+09,4.260214e+10,0.658949,B
5,SKU 6,131255.0,34867.0,4.576468e+09,4.717860e+10,0.729736,B
2,SKU 3,106396.0,40218.0,4.279034e+09,5.145764e+10,0.795922,B
10,SKU 11,112246.0,35468.0,3.981141e+09,5.543878e+10,0.857501,B
1,SKU 2,108196.0,35621.0,3.854050e+09,5.929283e+10,0.917113,C


## Clasificación ABC ventas (Precio x Demanda)

In [606]:
data

,Tipo,Familias,SKUS,Cargo de Mantener una unidad en un año,Rentabilidad,Total Demanda,Promedio semana,Desv. Estándar,Cv,Costo unitario,Costo Ordenar,Tasa Pdn (unid por semana)
0,compra y venta,Familia 1,SKU 1,0.18,0.18,100571.0,1117.455556,548.693672,0.491021,26478.0,3415860.0,NaN
1,compra y venta,NaN,SKU 2,0.18,0.25,108196.0,1202.177778,598.309990,0.497688,35621.0,5124610.0,NaN
2,compra y venta,NaN,SKU 3,0.18,0.22,106396.0,1182.177778,603.645267,0.510621,40218.0,2471600.0,NaN
3,Manufacturado,Familia 2,SKU 4,0.26,0.26,131526.0,1461.400000,274.140774,0.187588,64570.0,3451200.0,1800.0
4,Manufacturado,NaN,SKU 5,0.26,0.25,128338.0,1425.977778,253.785275,0.177973,56478.0,4023600.0,2000.0
5,Manufacturado,NaN,SKU 6,0.26,0.3,131255.0,1458.388889,281.232548,0.192838,34867.0,3845650.0,1700.0
6,compra y venta,Familia 3,SKU 7,0.20,0.15,120159.0,1335.100000,456.989761,0.342289,56416.0,6451870.0,NaN
7,compra y venta,NaN,SKU 8,0.20,0.21,123184.0,1368.711111,501.595393,0.366473,64179.0,4781900.0,NaN
8,compra y venta,NaN,SKU 9,0.20,0.25,120719.0,1341.322222,413.133431,0.308005,42351.0,2345640.0,NaN
9,Manufacturado,Familia 4,SKU 10,0.25,0.28,113177.0,1257.522222,434.914669,0.345850,62415.0,7845000.0,1500.0


In [607]:
dataSub = data[["SKUS",'Total Demanda','Costo unitario ',"Rentabilidad" ]]

In [608]:
dataSub["Precio"] =  data['Costo unitario '] * data["Rentabilidad"] +  data['Costo unitario ']

<ipython-input-608-1a4cc0e08882>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSub["Precio"] =  data['Costo unitario '] * data["Rentabilidad"] +  data['Costo unitario ']


In [609]:
dataSub = dataSub.sort_values(by="Precio", ascending=False)

In [610]:
totalPrecios = dataSub["Precio"].sum()
totalPrecios

673714.1900000002

In [611]:
dataSub["Precios Acumulados"] = dataSub["Precio"].cumsum()

In [612]:
dataSub

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Precio,Precios Acumulados
3,SKU 4,131526.0,64570.0,0.26,81358.2,81358.2
9,SKU 10,113177.0,62415.0,0.28,79891.2,161249.4
7,SKU 8,123184.0,64179.0,0.21,77656.59,238905.99
4,SKU 5,128338.0,56478.0,0.25,70597.5,309503.49
6,SKU 7,120159.0,56416.0,0.15,64878.4,374381.89
8,SKU 9,120719.0,42351.0,0.25,52938.75,427320.64
2,SKU 3,106396.0,40218.0,0.22,49065.96,476386.6
10,SKU 11,112246.0,35468.0,0.3,46108.4,522495.0
5,SKU 6,131255.0,34867.0,0.3,45327.1,567822.1
1,SKU 2,108196.0,35621.0,0.25,44526.25,612348.35


In [613]:
# Calcular el porcentaje de contribución de cada SKU al total de costo
dataSub['Porcentaje Contribución'] = (dataSub['Precios Acumulados'] / totalPrecios )

In [614]:
dataSub = dataSub.sort_values(by='Precios Acumulados', ascending=True)
dataSub

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Precio,Precios Acumulados,Porcentaje Contribución
3,SKU 4,131526.0,64570.0,0.26,81358.2,81358.2,0.120761
9,SKU 10,113177.0,62415.0,0.28,79891.2,161249.4,0.239344
7,SKU 8,123184.0,64179.0,0.21,77656.59,238905.99,0.35461
4,SKU 5,128338.0,56478.0,0.25,70597.5,309503.49,0.459399
6,SKU 7,120159.0,56416.0,0.15,64878.4,374381.89,0.555698
8,SKU 9,120719.0,42351.0,0.25,52938.75,427320.64,0.634276
2,SKU 3,106396.0,40218.0,0.22,49065.96,476386.6,0.707105
10,SKU 11,112246.0,35468.0,0.3,46108.4,522495.0,0.775544
5,SKU 6,131255.0,34867.0,0.3,45327.1,567822.1,0.842823
1,SKU 2,108196.0,35621.0,0.25,44526.25,612348.35,0.908914


In [615]:
dataSub['Class'] = dataSub['Porcentaje Contribución'].apply(ABC)

In [616]:
dataSub

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Precio,Precios Acumulados,Porcentaje Contribución,Class
3,SKU 4,131526.0,64570.0,0.26,81358.2,81358.2,0.120761,A
9,SKU 10,113177.0,62415.0,0.28,79891.2,161249.4,0.239344,A
7,SKU 8,123184.0,64179.0,0.21,77656.59,238905.99,0.35461,A
4,SKU 5,128338.0,56478.0,0.25,70597.5,309503.49,0.459399,A
6,SKU 7,120159.0,56416.0,0.15,64878.4,374381.89,0.555698,B
8,SKU 9,120719.0,42351.0,0.25,52938.75,427320.64,0.634276,B
2,SKU 3,106396.0,40218.0,0.22,49065.96,476386.6,0.707105,B
10,SKU 11,112246.0,35468.0,0.3,46108.4,522495.0,0.775544,B
5,SKU 6,131255.0,34867.0,0.3,45327.1,567822.1,0.842823,B
1,SKU 2,108196.0,35621.0,0.25,44526.25,612348.35,0.908914,C


## Clasificación ABC margen (Ganancia x Demanda)

In [617]:
dataSub2 = data[["SKUS",'Total Demanda','Costo unitario ',"Rentabilidad" ]]

In [618]:
dataSub2["Margen total acumulado por sku"] =  data['Costo unitario '] * data["Rentabilidad"] * data['Total Demanda']

<ipython-input-618-ae5e82fb92d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSub2["Margen total acumulado por sku"] =  data['Costo unitario '] * data["Rentabilidad"] * data['Total Demanda']


In [619]:
totalMargen = dataSub2["Margen total acumulado por sku"].sum()
totalMargen

15497852164.31

In [620]:
dataSub2 = dataSub2.sort_values(by="Margen total acumulado por sku", ascending=False)

In [621]:
dataSub2

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Margen total acumulado por sku
3,SKU 4,131526.0,64570.0,0.26,2208084793.2
9,SKU 10,113177.0,62415.0,0.28,1977903887.4
4,SKU 5,128338.0,56478.0,0.25,1812068391.0
7,SKU 8,123184.0,64179.0,0.21,1660223446.56
5,SKU 6,131255.0,34867.0,0.3,1372940425.5
8,SKU 9,120719.0,42351.0,0.25,1278142592.25
10,SKU 11,112246.0,35468.0,0.3,1194342338.4
6,SKU 7,120159.0,56416.0,0.15,1016833521.6
1,SKU 2,108196.0,35621.0,0.25,963512429.0
2,SKU 3,106396.0,40218.0,0.22,941387552.16


In [622]:
dataSub2["Margenes acumulados"] = dataSub2["Margen total acumulado por sku"].cumsum()

In [623]:
dataSub2

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Margen total acumulado por sku,Margenes acumulados
3,SKU 4,131526.0,64570.0,0.26,2208084793.2,2208084793.2
9,SKU 10,113177.0,62415.0,0.28,1977903887.4,4185988680.6
4,SKU 5,128338.0,56478.0,0.25,1812068391.0,5998057071.6
7,SKU 8,123184.0,64179.0,0.21,1660223446.56,7658280518.16
5,SKU 6,131255.0,34867.0,0.3,1372940425.5,9031220943.66
8,SKU 9,120719.0,42351.0,0.25,1278142592.25,10309363535.91
10,SKU 11,112246.0,35468.0,0.3,1194342338.4,11503705874.309999
6,SKU 7,120159.0,56416.0,0.15,1016833521.6,12520539395.91
1,SKU 2,108196.0,35621.0,0.25,963512429.0,13484051824.91
2,SKU 3,106396.0,40218.0,0.22,941387552.16,14425439377.07


In [624]:
# Calcular el porcentaje de contribución de cada SKU al total de costo
dataSub2['Porcentaje Contribución'] = (dataSub2['Margenes acumulados'] / totalMargen )

In [625]:
dataSub2['Class'] = dataSub2['Porcentaje Contribución'].apply(ABC)

In [626]:
dataSub2

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Margen total acumulado por sku,Margenes acumulados,Porcentaje Contribución,Class
3,SKU 4,131526.0,64570.0,0.26,2208084793.2,2208084793.2,0.142477,A
9,SKU 10,113177.0,62415.0,0.28,1977903887.4,4185988680.6,0.270101,A
4,SKU 5,128338.0,56478.0,0.25,1812068391.0,5998057071.6,0.387025,A
7,SKU 8,123184.0,64179.0,0.21,1660223446.56,7658280518.16,0.494151,A
5,SKU 6,131255.0,34867.0,0.3,1372940425.5,9031220943.66,0.58274,B
8,SKU 9,120719.0,42351.0,0.25,1278142592.25,10309363535.91,0.665212,B
10,SKU 11,112246.0,35468.0,0.3,1194342338.4,11503705874.309999,0.742277,B
6,SKU 7,120159.0,56416.0,0.15,1016833521.6,12520539395.91,0.807889,B
1,SKU 2,108196.0,35621.0,0.25,963512429.0,13484051824.91,0.870059,B
2,SKU 3,106396.0,40218.0,0.22,941387552.16,14425439377.07,0.930802,C


# Clasificación ABC Inventarios

## Clasificación ABC Inventario x Demanda

## Analisis comparativo

* Clasificación ABC - Ventas - Costos

In [627]:
dataSubset

,SKUS,Total Demanda,Costo unitario,Valor Total costos por sku,Costos Acumuludos,Porcentaje de costos acumulados,Class
3,SKU 4,131526.0,64570.0,8.492634e+09,8.492634e+09,0.131360,A
7,SKU 8,123184.0,64179.0,7.905826e+09,1.639846e+10,0.253644,A
4,SKU 5,128338.0,56478.0,7.248274e+09,2.364673e+10,0.365756,A
9,SKU 10,113177.0,62415.0,7.063942e+09,3.071068e+10,0.475018,A
6,SKU 7,120159.0,56416.0,6.778890e+09,3.748957e+10,0.579871,B
8,SKU 9,120719.0,42351.0,5.112570e+09,4.260214e+10,0.658949,B
5,SKU 6,131255.0,34867.0,4.576468e+09,4.717860e+10,0.729736,B
2,SKU 3,106396.0,40218.0,4.279034e+09,5.145764e+10,0.795922,B
10,SKU 11,112246.0,35468.0,3.981141e+09,5.543878e+10,0.857501,B
1,SKU 2,108196.0,35621.0,3.854050e+09,5.929283e+10,0.917113,C


* Clasificación ABC - Ventas - Precios

In [628]:
dataSub

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Precio,Precios Acumulados,Porcentaje Contribución,Class
3,SKU 4,131526.0,64570.0,0.26,81358.2,81358.2,0.120761,A
9,SKU 10,113177.0,62415.0,0.28,79891.2,161249.4,0.239344,A
7,SKU 8,123184.0,64179.0,0.21,77656.59,238905.99,0.35461,A
4,SKU 5,128338.0,56478.0,0.25,70597.5,309503.49,0.459399,A
6,SKU 7,120159.0,56416.0,0.15,64878.4,374381.89,0.555698,B
8,SKU 9,120719.0,42351.0,0.25,52938.75,427320.64,0.634276,B
2,SKU 3,106396.0,40218.0,0.22,49065.96,476386.6,0.707105,B
10,SKU 11,112246.0,35468.0,0.3,46108.4,522495.0,0.775544,B
5,SKU 6,131255.0,34867.0,0.3,45327.1,567822.1,0.842823,B
1,SKU 2,108196.0,35621.0,0.25,44526.25,612348.35,0.908914,C


* Clasificación ABC - Ventas - Margen

In [629]:
dataSub2

,SKUS,Total Demanda,Costo unitario,Rentabilidad,Margen total acumulado por sku,Margenes acumulados,Porcentaje Contribución,Class
3,SKU 4,131526.0,64570.0,0.26,2208084793.2,2208084793.2,0.142477,A
9,SKU 10,113177.0,62415.0,0.28,1977903887.4,4185988680.6,0.270101,A
4,SKU 5,128338.0,56478.0,0.25,1812068391.0,5998057071.6,0.387025,A
7,SKU 8,123184.0,64179.0,0.21,1660223446.56,7658280518.16,0.494151,A
5,SKU 6,131255.0,34867.0,0.3,1372940425.5,9031220943.66,0.58274,B
8,SKU 9,120719.0,42351.0,0.25,1278142592.25,10309363535.91,0.665212,B
10,SKU 11,112246.0,35468.0,0.3,1194342338.4,11503705874.309999,0.742277,B
6,SKU 7,120159.0,56416.0,0.15,1016833521.6,12520539395.91,0.807889,B
1,SKU 2,108196.0,35621.0,0.25,963512429.0,13484051824.91,0.870059,B
2,SKU 3,106396.0,40218.0,0.22,941387552.16,14425439377.07,0.930802,C


* Clasificación ABC - Inventarios -

* Analisis comparativo

In [630]:
df_clasificaciones = pd.DataFrame({
    'SKUS': range(1, 13),  # Filas de 1 a 12 SKUs
    'Costos': dataSubset.Class,
    'Precios': dataSub.Class,
    'Margen': dataSub2.Class
})

In [631]:
df_clasificaciones

,SKUS,Costos,Precios,Margen
0,1,C,C,C
1,2,C,C,B
2,3,B,B,C
3,4,A,A,A
4,5,A,A,A
5,6,B,B,B
6,7,B,B,B
7,8,A,A,A
8,9,B,B,B
9,10,A,A,A


# Definir base de datos y parametros

In [632]:
DataFrameCharacterization = data

DataFrameCharacterization

,Tipo,Familias,SKUS,Cargo de Mantener una unidad en un año,Rentabilidad,Total Demanda,Promedio semana,Desv. Estándar,Cv,Costo unitario,Costo Ordenar,Tasa Pdn (unid por semana)
0,compra y venta,Familia 1,SKU 1,0.18,0.18,100571.0,1117.455556,548.693672,0.491021,26478.0,3415860.0,NaN
1,compra y venta,NaN,SKU 2,0.18,0.25,108196.0,1202.177778,598.309990,0.497688,35621.0,5124610.0,NaN
2,compra y venta,NaN,SKU 3,0.18,0.22,106396.0,1182.177778,603.645267,0.510621,40218.0,2471600.0,NaN
3,Manufacturado,Familia 2,SKU 4,0.26,0.26,131526.0,1461.400000,274.140774,0.187588,64570.0,3451200.0,1800.0
4,Manufacturado,NaN,SKU 5,0.26,0.25,128338.0,1425.977778,253.785275,0.177973,56478.0,4023600.0,2000.0
5,Manufacturado,NaN,SKU 6,0.26,0.3,131255.0,1458.388889,281.232548,0.192838,34867.0,3845650.0,1700.0
6,compra y venta,Familia 3,SKU 7,0.20,0.15,120159.0,1335.100000,456.989761,0.342289,56416.0,6451870.0,NaN
7,compra y venta,NaN,SKU 8,0.20,0.21,123184.0,1368.711111,501.595393,0.366473,64179.0,4781900.0,NaN
8,compra y venta,NaN,SKU 9,0.20,0.25,120719.0,1341.322222,413.133431,0.308005,42351.0,2345640.0,NaN
9,Manufacturado,Familia 4,SKU 10,0.25,0.28,113177.0,1257.522222,434.914669,0.345850,62415.0,7845000.0,1500.0


In [633]:
DataFrameInventory = pd.read_excel("/content/drive/MyDrive/Operaciones I/Base de datos/Datos SKUS.xlsx")

DataFrameInventory

,SKU,Semana,Demanda,Inv Final,Pedido,Recibo
0,SKU1,1,1376,6475,NaN,NaN
1,SKU1,2,1284,5191,NaN,NaN
2,SKU1,3,983,4208,6000.0,NaN
3,SKU1,4,1740,2468,NaN,NaN
4,SKU1,5,1442,1026,NaN,6000.0
...,...,...,...,...,...,...
1075,SKU12,86,690,10872,NaN,NaN
1076,SKU12,87,675,10197,NaN,NaN
1077,SKU12,88,739,9458,NaN,NaN
1078,SKU12,89,499,8959,NaN,NaN


# (KPI) Key Performance Indicator by SKUS. (ORIGINAL)

Key Performance Indicator (Indicador Clave de Rendimiento). Es una métrica cuantificable que se utiliza para evaluar el éxito de una organización, empleado, equipo o proceso en el logro de objetivos clave de rendimiento. Los KPIs son herramientas fundamentales en la gestión empresarial, ya que proporcionan una medida focal para el rendimiento operativo o estratégico y permiten a las empresas y a sus partes interesadas evaluar la eficacia de sus actividades en relación con sus objetivos definidos.

In [634]:
DataFrameCharacterization

,Tipo,Familias,SKUS,Cargo de Mantener una unidad en un año,Rentabilidad,Total Demanda,Promedio semana,Desv. Estándar,Cv,Costo unitario,Costo Ordenar,Tasa Pdn (unid por semana)
0,compra y venta,Familia 1,SKU 1,0.18,0.18,100571.0,1117.455556,548.693672,0.491021,26478.0,3415860.0,NaN
1,compra y venta,NaN,SKU 2,0.18,0.25,108196.0,1202.177778,598.309990,0.497688,35621.0,5124610.0,NaN
2,compra y venta,NaN,SKU 3,0.18,0.22,106396.0,1182.177778,603.645267,0.510621,40218.0,2471600.0,NaN
3,Manufacturado,Familia 2,SKU 4,0.26,0.26,131526.0,1461.400000,274.140774,0.187588,64570.0,3451200.0,1800.0
4,Manufacturado,NaN,SKU 5,0.26,0.25,128338.0,1425.977778,253.785275,0.177973,56478.0,4023600.0,2000.0
5,Manufacturado,NaN,SKU 6,0.26,0.3,131255.0,1458.388889,281.232548,0.192838,34867.0,3845650.0,1700.0
6,compra y venta,Familia 3,SKU 7,0.20,0.15,120159.0,1335.100000,456.989761,0.342289,56416.0,6451870.0,NaN
7,compra y venta,NaN,SKU 8,0.20,0.21,123184.0,1368.711111,501.595393,0.366473,64179.0,4781900.0,NaN
8,compra y venta,NaN,SKU 9,0.20,0.25,120719.0,1341.322222,413.133431,0.308005,42351.0,2345640.0,NaN
9,Manufacturado,Familia 4,SKU 10,0.25,0.28,113177.0,1257.522222,434.914669,0.345850,62415.0,7845000.0,1500.0


In [635]:
import pandas as pd

# Contar los pedidos para cada SKU donde 'Pedido' es mayor que 0
pedido_counts = DataFrameInventory[DataFrameInventory['Pedido'] > 0].groupby('SKU').size().reset_index(name='Numero de pedidos')

import pandas as pd

# Suponiendo que pedido_counts es tu DataFrame
# Extraer el número después de 'SKU'
pedido_counts['SKU Number'] = pedido_counts['SKU'].str.extract('(\d+)').astype(int)
# Ordenar el DataFrame por la columna 'SKU Number'
pedido_counts = pedido_counts.sort_values(by='SKU Number')

# Opcional: Eliminar la columna 'SKU Number' si ya no se necesita
pedido_counts = pedido_counts.drop(columns=['SKU Number'])


pedido_counts

,SKU,Numero de pedidos
0,SKU1,17
4,SKU2,10
5,SKU3,12
6,SKU4,13
7,SKU5,13
8,SKU6,9
9,SKU7,15
10,SKU8,12
11,SKU9,14
1,SKU10,18


In [636]:
# Se calcula el costo de pedir

DataFrameCharacterization["Costo Total de Pedir (90 semanas)"] = pedido_counts['Numero de pedidos'] * DataFrameCharacterization['Costo Ordenar']

In [637]:
# Se reajusta la ventana de tiempo del cargo de mantener una unidad en un año a 90 semanas

DataFrameCharacterization['Cargo de Mantener una unidad en un año'] = DataFrameCharacterization['Cargo de Mantener una unidad en un año'] * (90 / 52)
DataFrameCharacterization = DataFrameCharacterization.rename(columns={'Cargo de Mantener una unidad en un año': 'Cargo de Mantener una unidad en 90 Semanas'})

In [638]:
# Se reajusta el nombre de la columna para ser más especifico a Inventario promedio del sku en 90 semanas

DataFrameCharacterization = DataFrameCharacterization.rename(columns={'Promedio semana': 'Inventario promedio (90 Semanas)'})

In [639]:
# Se calcula el costo de manteminimiento total

DataFrameCharacterization['Costo Total de Mantenimiento (90 semanas)'] = DataFrameCharacterization['Inventario promedio (90 Semanas)'] * DataFrameCharacterization['Costo unitario '] * DataFrameCharacterization['Cargo de Mantener una unidad en 90 Semanas'] / 100

In [640]:
# Se calcula los costos totales

DataFrameCharacterization['Costo Totales'] = DataFrameCharacterization['Costo Total de Pedir (90 semanas)'] + DataFrameCharacterization['Costo Total de Mantenimiento (90 semanas)']

In [641]:
DataFrameCharacterization

,Tipo,Familias,SKUS,Cargo de Mantener una unidad en 90 Semanas,Rentabilidad,Total Demanda,Inventario promedio (90 Semanas),Desv. Estándar,Cv,Costo unitario,Costo Ordenar,Tasa Pdn (unid por semana),Costo Total de Pedir (90 semanas),Costo Total de Mantenimiento (90 semanas),Costo Totales
0,compra y venta,Familia 1,SKU 1,0.311538,0.18,100571.0,1117.455556,548.693672,0.491021,26478.0,3415860.0,NaN,58069620.0,92177.963238,5.816180e+07
1,compra y venta,NaN,SKU 2,0.311538,0.25,108196.0,1202.177778,598.309990,0.497688,35621.0,5124610.0,NaN,92242980.0,133409.413246,9.237639e+07
2,compra y venta,NaN,SKU 3,0.311538,0.22,106396.0,1182.177778,603.645267,0.510621,40218.0,2471600.0,NaN,27187600.0,148120.419046,2.733572e+07
3,Manufacturado,Familia 2,SKU 4,0.450000,0.26,131526.0,1461.400000,274.140774,0.187588,64570.0,3451200.0,1800.0,75926400.0,424631.691000,7.635103e+07
4,Manufacturado,NaN,SKU 5,0.450000,0.25,128338.0,1425.977778,253.785275,0.177973,56478.0,4023600.0,2000.0,40236000.0,362413.678200,4.059841e+07
5,Manufacturado,NaN,SKU 6,0.450000,0.3,131255.0,1458.388889,281.232548,0.192838,34867.0,3845650.0,1700.0,46147800.0,228823.404250,4.637662e+07
6,compra y venta,Familia 3,SKU 7,0.346154,0.15,120159.0,1335.100000,456.989761,0.342289,56416.0,6451870.0,NaN,83874310.0,260726.544000,8.413504e+07
7,compra y venta,NaN,SKU 8,0.346154,0.21,123184.0,1368.711111,501.595393,0.366473,64179.0,4781900.0,NaN,62164700.0,304070.228308,6.246877e+07
8,compra y venta,NaN,SKU 9,0.346154,0.25,120719.0,1341.322222,413.133431,0.308005,42351.0,2345640.0,NaN,21110760.0,196637.321885,2.130740e+07
9,Manufacturado,Familia 4,SKU 10,0.432692,0.28,113177.0,1257.522222,434.914669,0.345850,62415.0,7845000.0,1500.0,117675000.0,339612.618029,1.180146e+08


# (KPI) Key Performance Indicator by SKUS. (EOQ Y EPQ)

## Calculo de EOQ Y EPQ

* Economic Order Quantity (EOQ)

In [642]:
import math
def eoq (D, S, h):
  return math.sqrt((2 * D * S) /  (h))

* Economic Production Quantity (EPQ)

In [643]:
import math
def epq(D, S, h, p):
    return math.sqrt(   (2 * D * S) / ((1 - (D / p)) * h)   )

In [644]:

def calculate_Annual_Demand(sku):
  demanda_promedio = (DataFrameInventory[DataFrameInventory['SKU'].isin([sku])]["Demanda"].sum())/90 #Se saca el promedio de la demanda total (90 semanas)
  return demanda_promedio

def fixed_Costs(sku):
  costo_por_ordenar = DataFrameCharacterization.loc[DataFrameCharacterization['SKUS'] == sku, "Costo Ordenar"].iloc[0]
  return costo_por_ordenar

def holding_Costs(sku):
    costo_por_mantener = DataFrameCharacterization.loc[DataFrameCharacterization['SKUS'] == sku, "Cargo de Mantener una unidad en 90 Semanas"].iloc[0]
    costo_unitario = unit_Cost(sku)
    return costo_por_mantener * costo_unitario # Se multiplica el costo por mantener un sku especifico por su costo unitario

def unit_Cost(sku):
    return DataFrameCharacterization.loc[DataFrameCharacterization['SKUS'] == sku, "Costo unitario "].iloc[0] * 1.73

def productionRate(sku):
  return DataFrameCharacterization.loc[DataFrameCharacterization['SKUS'] == sku, "Tasa Pdn (unid por semana)"].iloc[0]*90

### Lot Sizing for a Single Product (Economic Order Quantity) (EOQ)

**1. Calcular el EOQ Del SKU 1, para 1 año (52 semanas)**

In [645]:
D = calculate_Annual_Demand("SKU1")

S = fixed_Costs("SKU 1")

h = holding_Costs("SKU 1")

c = unit_Cost("SKU 1")

In [646]:
print("Demanda total: ", D ,

      " Costo por ordenar: ", S ,

      " Costos inventario: ", h ,

      " Costo unitario: ", c)

Demanda total:  1117.4555555555555  Costo por ordenar:  3415860.0  Costos inventario:  14270.623615384617  Costo unitario:  45806.94


In [647]:
sku1_eoq = eoq(D,S,h)
print(" EOQ : ", sku1_eoq)

 EOQ :  731.4062762768415


***2. Calcular el EOQ Del SKU 2, para 1 año (52 semanas)***

In [648]:
D2 = calculate_Annual_Demand("SKU2")

S2 = fixed_Costs("SKU 2")

h2 = holding_Costs("SKU 2")

c2 = unit_Cost("SKU 2")

In [649]:
print("Demanda total: ", D2 ,

      " Costo por ordenar: ", S2 ,

      " Costos inventario: ", h2 ,

      " Costo unitario: ", c2)

Demanda total:  1202.1777777777777  Costo por ordenar:  5124610.0  Costos inventario:  19198.34896153846  Costo unitario:  61624.33


In [650]:
sku2_eoq = eoq(D2,S2,h2)
print(" EOQ : ", sku2_eoq)

 EOQ :  801.120444221653


***3. Calcular el EOQ Del SKU 3, para 1 año (52 semanas)***

In [651]:
D3 = calculate_Annual_Demand("SKU3")

S3 = fixed_Costs("SKU 3")

h3 = holding_Costs("SKU 3")

c3 = unit_Cost("SKU 3")

In [652]:
print("Demanda total: ", D3,

      " Costo por ordenar: ", S3,

      " Costos inventario: ", h3,

      " Costo unitario: ", c3)

Demanda total:  1182.1777777777777  Costo por ordenar:  2471600.0  Costos inventario:  21675.955153846153  Costo unitario:  69577.14


In [653]:
sku3_eoq = eoq(D3,S3,h3)
print(" EOQ : ", sku3_eoq)

 EOQ :  519.225920425698


***7. Calcular el EOQ Del SKU 7, para 1 año (52 semanas)***

In [654]:
D7 = calculate_Annual_Demand("SKU7")

S7 = fixed_Costs("SKU 7")

h7 = holding_Costs("SKU 7")

c7 = unit_Cost("SKU 7")

In [655]:
print("Demanda total: ", D7,

      " Costo por ordenar: ", S7,

      " Costos inventario: ", h7,

      " Costo unitario: ", c7)

Demanda total:  1335.1  Costo por ordenar:  6451870.0  Costos inventario:  33784.50461538462  Costo unitario:  97599.68


In [656]:
sku7_eoq = eoq(D7,S7,h7)

print(" EOQ : ", sku7_eoq)


 EOQ :  714.0948849403779


***8. Calcular el EOQ Del SKU 3, para 1 año (52 semanas)***

In [657]:
D8 = calculate_Annual_Demand("SKU8")

S8 = fixed_Costs("SKU 8")

h8 = holding_Costs("SKU 8")

c8 = unit_Cost("SKU 8")

In [658]:
print("Demanda total: ", D8 ,

      " Costo por ordenar: ", S8,

      " Costos inventario: ", h8 ,

      " Costo unitario: ", c8)

Demanda total:  1368.7111111111112  Costo por ordenar:  4781900.0  Costos inventario:  38433.34730769231  Costo unitario:  111029.67


In [659]:
sku8_eoq = eoq(D8,S8,h8)

print(" EOQ : ", sku8_eoq)


 EOQ :  583.6023525811536


***9. Calcular el EOQ Del SKU 3, para 1 año (52 semanas)***

In [660]:
D9 = calculate_Annual_Demand("SKU9")

S9 = fixed_Costs("SKU 9")

h9 = holding_Costs("SKU 9")

c9 = unit_Cost("SKU 9")

In [661]:
print("Demanda total: ", D9 ,

      " Costo por ordenar: ", S9,

      " Costos inventario: ", h9 ,

      " Costo unitario: ", c9)

Demanda total:  1341.3222222222223  Costo por ordenar:  2345640.0  Costos inventario:  25361.733461538464  Costo unitario:  73267.23


In [662]:
sku9_eoq = eoq(D9,S9,h9)
print(" EOQ : ", sku9_eoq)

 EOQ :  498.1071436035339


### Lot Sizing for a Single Product (Economic production Quantity) (EPQ)

**1. Calcular el EPQ Del SKU 4, para 1 año (52 semanas)**

In [663]:
D4 = calculate_Annual_Demand("SKU4")

S4 = fixed_Costs("SKU 4")

h4 = holding_Costs("SKU 4")

c4 = unit_Cost("SKU 4")

p4 = productionRate("SKU 4")

In [664]:
print("Demanda total: ", D4 ,

      " Costo por ordenar: ", S4 ,

      " Costos inventario: ", h4 ,

      " Costo unitario: ", c4 ,

      " Tasa de producción: ", p4)

Demanda total:  1461.4  Costo por ordenar:  3451200.0  Costos inventario:  50267.745  Costo unitario:  111706.1  Tasa de producción:  162000.0


In [665]:
sku4_epq = epq(D4,S4,h4,p4)

print(" EPQ : ", sku4_epq)


 EPQ :  449.99499472913493


**2. Calcular el EPQ Del SKU 5, para 1 año (52 semanas)**

In [666]:
D5 = calculate_Annual_Demand("SKU5")

S5 = fixed_Costs("SKU 5")

h5 = holding_Costs("SKU 5")

c5 = unit_Cost("SKU 5")

p5 = productionRate("SKU 5")

In [667]:
print("Demanda total: ", D5 ,

      " Costo por ordenar: ", S5 ,

      " Costos inventario: ", h5 ,

      " Costo unitario: ", c5,

      " Tasa de producción: ", p5)

Demanda total:  1425.9777777777779  Costo por ordenar:  4023600.0  Costos inventario:  43968.123  Costo unitario:  97706.94  Tasa de producción:  180000.0


In [668]:
sku5_epq = epq(D5,S5,h5,p5)
print(" EPQ : ", sku5_epq)

 EPQ :  512.9049914392754


**3. Calcular el EPQ Del SKU 6, para 1 año (52 semanas)**

In [669]:
D6 = calculate_Annual_Demand("SKU6")

S6 = fixed_Costs("SKU 6")

h6 = holding_Costs("SKU 6")

c6 = unit_Cost("SKU 6")

p6 = productionRate("SKU 6")

In [670]:
print("Demanda total: ", D6,

      " Costo por ordenar: ", S6,

      " Costos inventario: ", h6,

      " Costo unitario: ", c6,

      " Tasa de producción: ", p6)

Demanda total:  1458.388888888889  Costo por ordenar:  3845650.0  Costos inventario:  27143.959499999997  Costo unitario:  60319.909999999996  Tasa de producción:  153000.0


In [671]:
sku6_epq = epq(D6,S6,h6,p6)
print(" EPQ : ", sku6_epq)

 EPQ :  645.9214621293783


**4. Calcular el EOQ Del SKU 10, para 1 año (52 semanas)**

In [672]:
D10 = calculate_Annual_Demand("SKU10")

S10 = fixed_Costs("SKU 10")

h10 = holding_Costs("SKU 10")

c10 = unit_Cost("SKU 10")

p10 = productionRate("SKU 10")

In [673]:
print("Demanda total: ", D10,

      " Costo por ordenar: ", S10,

      " Costos inventario: ", h10,

      " Costo unitario: ", c10,

      " Tasa de producción: ", p10)

Demanda total:  1257.5222222222221  Costo por ordenar:  7845000.0  Costos inventario:  46721.22836538462  Costo unitario:  107977.95  Tasa de producción:  135000.0


In [674]:
sku10_epq = epq(D6,S6,h6,p6)

print(" EPQ : ", sku10_epq)

 EPQ :  645.9214621293783


**5. Calcular el EOQ Del SKU 11, para 1 año (52 semanas)**

In [675]:
D11 = calculate_Annual_Demand("SKU11")

S11 = fixed_Costs("SKU 11")

h11 = holding_Costs("SKU 11")

c11 = unit_Cost("SKU 11")

p11 = productionRate("SKU 11")

In [676]:
print("Demanda total: ", D11,

      " Costo por ordenar: ", S11,

      " Costos inventario: ", h11,

      " Costo unitario: ", c11,

      " Tasa de producción: ", p11)

Demanda total:  1247.1777777777777  Costo por ordenar:  4567180.0  Costos inventario:  26549.84423076923  Costo unitario:  61359.64  Tasa de producción:  198000.0


In [677]:
sku11_epq = epq(D11,S11,h11,p11)
print(" EPQ : ", sku11_epq)

 EPQ :  657.1195151336029


**6. Calcular el EOQ Del SKU 12, para 1 año (52 semanas)**

In [678]:
D12 = calculate_Annual_Demand("SKU12")

S12 = fixed_Costs("SKU 12")

h12 = holding_Costs("SKU 12")

c12 = unit_Cost("SKU 12")

p12 = productionRate("SKU 12")

In [679]:
print("Demanda total: ", D12,

      " Costo por ordenar: ", S12,

      " Costos inventario: ", h12,

      " Costo unitario: ", c12,

      " Tasa de producción: ", p12)

Demanda total:  1213.2  Costo por ordenar:  3654130.0  Costos inventario:  18481.889423076922  Costo unitario:  42713.7  Tasa de producción:  225000.0


In [680]:
sku12_epq = epq(D12,S12,h12,p12)
print(" EPQ : ", sku12_epq)

 EPQ :  694.502858756653


## Simulación de reabastecimiento por EQO y EPQ

4.	Retomar las 90 semanas de demanda y simular la aplicación de cantidades de reabastecimiento según valores calculados de EOQ y EPQ, asumiendo reabastecimiento automático (lead time igual a 0). Calcular el costo promedio semanal y revisar el impacto en la gestión de los inventarios de los SKUs con demandas no estables. Calcular los tiempos promedio de ciclos y la tasa promedio de rotación de los inventarios.

In [681]:
dataInventory = pd.read_excel("/content/drive/MyDrive/Operaciones I/Base de datos/datos de invetarios.xlsx")

In [682]:
dataInventory

,SKU,Semana,Demanda,Inv Final,Pedido,Recibo
0,SKU1,1,1376,6475,0,0
1,SKU1,2,1284,5191,0,0
2,SKU1,3,983,4208,0,0
3,SKU1,4,1740,2468,0,0
4,SKU1,5,1442,1026,0,0
...,...,...,...,...,...,...
1075,SKU12,86,690,-99128,0,0
1076,SKU12,87,675,-99803,0,0
1077,SKU12,88,739,-100542,0,0
1078,SKU12,89,499,-101041,0,0


**Paso 1: Simulación del Reabastecimiento.**


Vamos a simular el reabastecimiento de inventario durante 90 semanas utilizando los valores de EOQ (Economic Order Quantity) y EPQ (Economic Production Quantity) para los diferentes SKUs. Se asumirá que el reabastecimiento es automático (lead time = 0), lo que significa que cualquier pedido realizado en una semana estará disponible de inmediato.

In [683]:
def simular_reabastecimiento(df, eoq_epq_dict):
    df['Pedido'] = 0
    df['Recibo'] = 0

    for index, row in df.iterrows():
        # Calcula la demanda de la semana y actualiza el inventario
        if index > 0:  # Asegura que usamos el inventario final de la semana anterior como inicial para la nueva
            df.at[index, 'Inv Final'] = df.at[index - 1, 'Inv Final']

        df.at[index, 'Inv Final'] -= row['Demanda']

        # Revisa si es necesario reabastecer
        if df.at[index, 'Inv Final'] <= 0:
            sku = row['SKU']
            order_quantity = eoq_epq_dict.get(sku, 0)
            df.at[index, 'Pedido'] = order_quantity
            df.at[index, 'Recibo'] = order_quantity
            # Asegurarnos de que el inventario se actualiza considerando el nuevo stock recibido
            df.at[index, 'Inv Final'] += order_quantity
    return df


# Asegurarse de que eoq_epq_dict contiene los valores correctos para cada SKU
eoq_epq_dict = {
    'SKU1': sku1_eoq,  # Ejemplo: Asumir que el EOQ para SKU1 es 1000
    'SKU2': sku2_eoq,
    'SKU3': sku3_eoq,
    'SKU4': sku4_epq,
    'SKU5': sku5_epq,
    'SKU6': sku6_epq,
    'SKU7': sku7_eoq,
    'SKU8': sku8_eoq,
    'SKU9': sku9_eoq,
    'SKU10': sku10_epq,
    'SKU11': sku11_epq,
    'SKU12': sku12_epq,# Ejemplo: Asumir que el EOQ para SKU2 es 800
    # Añade las entradas apropiadas para otros SKUs
}

# Llamar a la función reabastecer y asignar el resultado a la variable df si se desea mantener los cambios
df = simular_reabastecimiento(dataInventory, eoq_epq_dict)

df.head(1000)

,SKU,Semana,Demanda,Inv Final,Pedido,Recibo
0,SKU1,1,1376,5099.000000,0.000000,0.000000
1,SKU1,2,1284,3815.000000,0.000000,0.000000
2,SKU1,3,983,2832.000000,0.000000,0.000000
3,SKU1,4,1740,1092.000000,0.000000,0.000000
4,SKU1,5,1442,381.406276,731.406276,731.406276
...,...,...,...,...,...,...
995,SKU12,6,1566,-688440.857668,694.502859,694.502859
996,SKU12,7,1652,-689398.354809,694.502859,694.502859
997,SKU12,8,1615,-690318.851950,694.502859,694.502859
998,SKU12,9,1848,-691472.349091,694.502859,694.502859


# (KPI) Key Performance Indicator by SKUS. (Dynamic Lot - sizing)

Para el paso 6 de aplicar métodos heurísticos y/o algoritmos de optimización de lot sizing a los SKUs con demandas irregulares durante 90 semanas, vamos a centrarnos en cómo implementar un enfoque heurístico común, el Método de Silver-Meal, y cómo calcular posteriormente el costo promedio semanal y la tasa promedio de rotación de los inventarios. Estos cálculos ayudarán a evaluar la efectividad de la estrategia de lot sizing aplicada.



## POQ

## W-W

## Heuristic

### **Aplicación del Método de Silver-Meal**

El método de Silver-Meal es una heurística de programación de lotes que busca minimizar el costo total por unidad de tiempo al decidir cuándo y cuánto reordenar. Es especialmente útil para demandas que no son constantes (irregulares).

#### Paso a Paso del Método de Silver-Meal

1. **Inicialización**: Empieza con el primer periodo como el inicio del lote.
2. **Cálculo de Costos**: Calcula el costo total por unidad de tiempo para cada periodo adicional agregado al lote.
3. **Decisión**: Si agregar otro periodo al lote reduce el costo promedio por unidad de tiempo, inclúyelo en el lote. Si no, termina el lote y comienza otro.

In [ ]:
import pandas as pd
import numpy as np

def silver_meal(df, costo_setup, costo_unitario):
    n = len(df)
    lot_sizes = [0]*n
    i = 0

    while i < n:
        min_cost_per_unit_time = float('inf')
        best_k = i
        cumulative_demand = 0
        for k in range(i, n):
            cumulative_demand += df.loc[k, 'Demanda']
            total_cost = costo_setup + (cumulative_demand * costo_unitario)
            average_cost_per_unit_time = total_cost / (k - i + 1)

            if average_cost_per_unit_time < min_cost_per_unit_time:
                min_cost_per_unit_time = average_cost_per_unit_time
                best_k = k
            else:
                break

        lot_size = df.loc[i:best_k, 'Demanda'].sum()
        for j in range(i, best_k + 1):
            lot_sizes[j] = lot_size

        i = best_k + 1

    df['Lot Size'] = lot_sizes
    return df

# Asumiendo que df es tu DataFrame con los SKUs con demandas irregulares
costo_setup = 100  # Costo fijo de hacer un pedido
costo_unitario = 5  # Costo por unidad de mantener el inventario

df_optimized = silver_meal(df, costo_setup, costo_unitario)


### **Calcular el Costo Promedio Semanal y la Tasa de Rotación**

Con los lotes ya optimizados, puedes calcular el costo promedio semanal de la siguiente manera

In [ ]:
def calcular_costos(df, costo_unitario, costo_setup):
    df['Costo Mantenimiento'] = df['Lot Size'] * costo_unitario
    df['Costo Pedido'] = (df['Lot Size'] > 0).astype(int) * costo_setup
    df['Costo Total'] = df['Costo Mantenimiento'] + df['Costo Pedido']

    return df['Costo Total'].mean()

costo_promedio_semanal = calcular_costos(df_optimized, costo_unitario, costo_setup)
print("Costo promedio semanal:", costo_promedio_semanal)

def calcular_tasa_rotacion(df):
    total_vendido = df['Demanda'].sum()
    inventario_promedio = df['Lot Size'].mean()

    return total_vendido / inventario_promedio if inventario_promedio != 0 else 0

tasa_rotacion = calcular_tasa_rotacion(df_optimized)
print("Tasa promedio de rotación:", tasa_rotacion)

Aplicar una heurística de lot sizing como Silver-Meal permite optimizar los tamaños de lote de reabastecimiento basados en la minimización de costos a lo largo del tiempo, lo que es crucial para manejar eficientemente inventarios con demandas irregulares. Posteriormente, el cálculo del costo promedio semanal y la tasa de rotación proporciona métricas clave para evaluar el rendimiento de la estrategia de inventario implementada.

# (KPI) Key Performance Indicator by SKUS. (Jointly replenishment based on : Total & Partial)

* reabastecimiento conjunto de los modelos M1 y M2 para los skus 1,2,3

* reabastecimiento conjunto de los modelos M1 y M2 para los skus 7,8,9